<a href="https://colab.research.google.com/github/yexf308/AppliedStatistics/blob/main/Homework/HW5/565Sp23HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pylab inline 
import pandas as pd
from scipy import linalg
from itertools import combinations
import scipy
import sklearn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

Populating the interactive namespace from numpy and matplotlib


$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$
# Homework 5
## Homework guideline
- The deadline is May 5th 10:30am. Submission after the deadline will not be graded. 


- Submit your work(your reasoning and your code) as a SINGLE .ipynb document. Please rename the document as _HW1_YOURNAME.ipynb_ (for example, _HW1_FELIX.ipynb_). You are responsible for checking that you have correctly submitted the correct document. If your code cannot run, you may receive NO point. 

- Please justify all short answers with a brief explanation. 



- You only use the Python packages included in the following cell. You are not allowed to use other advanced package or modules unless you are permitted to. 

- In your final submission include the plots produced by the unedited code as presented below, as well as any additional plots produced after editing the code during the course of a problem. You may find it necessary to copy/paste relevant code into additional cells to accomplish this.

- Feel free to use the lecture notes and other resources. But you
must understand, write, and hand in your own answers. In addition, you must write and submit
your own code in the programming part of the assignment (we may run your code).
If you copy someone else homework solution, both of you may receive ZERO point. 


- Colab is preferred. However, if you use Anaconda, please download the .mat file locally and save it to the same folder as this homework file. 



**Collaboration:** List the names of all people you collaborated with and for which question(s). This is important!

# Q1: Flight Delay
Your task is to beat at least 2 benchmarks in this question. You may use any machine learning algorithm you like, and may tune
it any way you wish. You may use the method and helper functions built into scikit_learn.  you do not need
to implement the method yourself, but may if you wish.

Here you won’t be provided with detailed instructions. We only give you a brief description of how the second benchmark was achieved using Xgboost. Hopefully, at this stage of the course, it's enough for you to take a quick look at the data in order to understand that this is the type of task where gradient boosting will perform well. Most likely it will be Xgboost, however, we’ve got plenty of categorical features here.

In [2]:
!wget https://raw.githubusercontent.com/yexf308/AppliedStatistics/main/Homework/HW5/flight_delays_train.csv?raw=true -O flight_delays_train.csv
!wget https://raw.githubusercontent.com/yexf308/AppliedStatistics/main/Homework/HW5/flight_delays_test.csv?raw=true -O flight_delays_test.csv


--2022-12-13 19:57:43--  https://raw.githubusercontent.com/yexf308/AppliedStatistics/main/Homework/HW5/flight_delays_train.csv?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488296 (3.3M) [text/plain]
Saving to: ‘flight_delays_train.csv’

flight_delays_train 100%[===================>]   3.33M  --.-KB/s    in 0.04s   

2022-12-13 19:57:43 (77.4 MB/s) - ‘flight_delays_train.csv’ saved [3488296/3488296]

--2022-12-13 19:57:43--  https://raw.githubusercontent.com/yexf308/AppliedStatistics/main/Homework/HW5/flight_delays_test.csv?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... con

In [3]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [4]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


### Benchmark 1
Given flight departure time, carrier's code, departure airport, destination location, and flight distance, you have to predict departure delay for more than 15 minutes. As the simplest benchmark, let's take Xgboost classifier and two features that are easiest to take: DepTime and Distance. 


In [6]:
X_train = train[['Distance', 'DepTime']].values
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = \
    train_test_split(X_train, y_train, 
                     test_size=0.3, random_state=17)

We'll train Xgboost with default parameters on part of data and estimate holdout [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). 



In [7]:
xgb_model = XGBClassifier(seed=17)

xgb_model.fit(X_train_part, y_train_part)
xgb_valid_pred = xgb_model.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, xgb_valid_pred)

0.6974401153652741

Now we do the same with the whole training set, make predictions to test set and form a submission file. This is how you set the first benchmark.

In [8]:
xgb_model.fit(X_train, y_train)
xgb_test_pred = xgb_model.predict_proba(X_test)[:, 1]

pd.Series(xgb_test_pred, 
          name='dep_delayed_15min').to_csv('xgb_2feat.csv', 
                                           index_label='id', header=True)

You can see the folder, there is a file called *xgb_2feat.csv* that you can download. 



---


## Q1.1: Implement Benchmark 2 (50pt)
The second benchmark is achieved as follows:

- Features ```Distance``` and ```DepTime``` were taken unchanged.

- A feature ```Flight``` was created from features ```Origin``` and ```Dest```.

- Features ```Month```, ```DayofMonth```, ```DayOfWeek```, ```UniqueCarrier``` and ```Flight``` were transformed with onehot encoder (LabelBinarizer). 

- Logistic regression and gradient boosting (xgboost) were trained. Xgboost hyperparameters were tuned via cross-validation. First, the hyperparameters responsible for model complexity were optimized, then the number of trees was fixed at 500 and learning step was tuned.

- Predicted probabilities were made via cross-validation using ```cross_val_predict```. A linear mixture of logistic regression and gradient boosting predictions was set in the form
$w_1 P_{\text{logit}} + (1-w_1) P_{\text{xgb}}$, where $P_{\text{logit}}$ is the probability predicted by the logistic regression and $P_{\text{xgb}}$ is the probability predicted by the xgboost, $w_1$ weight was selected manually.

- A similar combination of predictions was made for test set. Be careful not to shuffle the test instances; the
order of the predictions must match the order of the test instances. 



**Please implement benchmark 2 algorithms.**

- Record estimated holdout ROC AUC. 

- Save your test data prediction to ```BenchMark2.csv```.

In [ ]:
# Your code starts here

# Your Solution:



---


## Q1.2: Training the Best Classifier (50pt)
Now, train the very best classifier for this problem to beat both benchmarks in holdout ROC AUC. Perhaps you might not want to follow the same steps as benchmark 2, and instead, let’s say, add a couple of good features and train a random forest of a thousand trees :)


- Record estimated holdout ROC AUC. 

- Save your test data prediction to ```Best.csv```.

- Write a brief paragraph (6–8 sentences max) describing the best machine learning classifier you found, its
optimal parameter settings (if any), and how you trained the model. (like what I write in benchmark 2)

In [ ]:
# Your code starts here

# Your Solution:





---



---

# Q2: (optional) Correction to your previous homework question
You may pick any one question in homework 1-4 that didn't perform well, and now you have the chance to correct your mistakes. If you successfully correct your mistakes, your previous grade will be replaced by the current score.

**State Your question that you want to correct:**

In [ ]:
# Your new code starts here

# Your New Solution: